In [1]:
# Implementation of the experiments in the paper SPLADE v2: Sparse Lexical and
# Expansion Model for Information Retrieval, (Thibault Formal, Carlos Lassance,
# Benjamin Piwowarski, Stéphane Clinchant), 2021
# https://arxiv.org/abs/2109.10086

import logging
from functools import lru_cache

from experimaestro.launcherfinder import find_launcher

from experimaestro import tag, setmeta, experiment
from xpmir.distributed import DistributedHook
from xpmir.letor.learner import Learner, ValidationListener
from xpmir.letor.schedulers import LinearWithWarmup
from xpmir.index.sparse import (
    SparseRetriever,
    SparseRetrieverIndexBuilder,
)
from xpmir.letor.distillation.pairwise import (
    DistillationPairwiseTrainer,
    MSEDifferenceLoss,
)
#from xpmir.papers.cli import paper_command
from xpmir.letor.samplers import PairwiseSampler
from xpmir.letor.trainers.batchwise import BatchwiseTrainer, SoftmaxCrossEntropy
from xpmir.letor.batchers import PowerAdaptativeBatcher
from xpmir.neural.dual import DenseDocumentEncoder, DenseQueryEncoder
from xpmir.letor.optim import (
    ParameterOptimizer,
    AdamW,
    get_optimizers,
)
from xpmir.rankers.standard import BM25
from xpmir.neural.splade import spladeV2_max, spladeV2_doc
from xpmir.papers.results import PaperResults
from xpmir.papers.splade.pipeline_trec import SPLADETRECCast2020Experiment
from xpmir.papers.splade.configuration_trec import SPLADE_TREC, Learner_TREC as LearnerConfig
from experimaestro.utils.jobs import jobmonitor
from pathlib import Path
import sys
from IPython.display import display, Markdown, Latex
import xpmir.letor.trainers.pairwise as pairwise

logging.basicConfig(level=logging.INFO)

# Espace pour les fichiers
libpath = Path(".") / "lib" / "xpmir_userlib"
libpath.mkdir(parents=True, exist_ok=True)
(libpath / "__init__.py").touch()
libpath = str(libpath.absolute().parent)
if libpath not in sys.path:
    sys.path.append(libpath)

In [2]:
from experimaestro.utils.jupyter import serverwidget
from xpmir.letor.optim import TensorboardService

tb = []
def init_xp(xp):
    # Évite que plus d'une tâche ne s'exécute à la fois|
    # TODO: use GLOBAL token
    xp.token = xp.current.token("main", 1)
    
    tb.clear()
    tb.append(xp.current.add_service(TensorboardService(xp.current.resultspath / "runs")))
    xp.current.setenv("PYTHONPATH", libpath)

xp = serverwidget("xp/ri", environment={"JAVA_HOME": "/usr/lib/jvm/java-11-openjdk-amd64"}, port=12500, hook=init_xp)

Button(description='Start the experimaestro server', style=ButtonStyle())

Output()

INFO:xpmir.letor.optim:tensorboard --logdir=/home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/xp/ri/xp/xp/ri/results/runs


Server started : http://localhost:12500/auth?xpm-token=763033d7784e4c9fa7620637c8326888


In [3]:
# Run by:
# $ xpmir papers splade spladeV2 --configuration config_name index/


class SPLADEIndex(SPLADETRECCast2020Experiment):
    """SPLADEv2 models"""

    cfg: SPLADE_TREC

    basemodel = BM25()

    def __init__(self, xp: experiment, cfg: SPLADE_TREC):
        super().__init__(xp, cfg)
        self.gpu_launcher_learner = find_launcher(cfg.learner.requirements)
        self.gpu_launcher_evaluate = find_launcher(cfg.evaluation.requirements)
        
    @lru_cache
    def get_optimizers(self, cfg: LearnerConfig):
        scheduler = (
            LinearWithWarmup(num_warmup_steps=cfg.num_warmup_steps)
            if cfg.scheduler
            else None
        )

        return get_optimizers(
            [
                ParameterOptimizer(
                    scheduler=scheduler,
                    optimizer=AdamW(lr=cfg.lr),
                )
            ]
        )

    def run(self):
        """SPLADE model (only indexing)"""

        cfg = self.cfg
        # -----Learning to rank component preparation part-----

        # Define the model and the flop loss for regularization
        # Model of class: DotDense()
        # The parameters are the regularization coeff for the query and document
        if cfg.learner.model == "splade_max":
            spladev2, flops = spladeV2_max(
                cfg.learner.lambda_q,
                cfg.learner.lambda_d,
                cfg.learner.lamdba_warmup_steps,
            )
        elif cfg.learner.model == "splade_doc":
            spladev2, flops = spladeV2_doc(
                cfg.learner.lambda_q,
                cfg.learner.lambda_d,
                cfg.learner.lamdba_warmup_steps,
            )
        else:
            raise NotImplementedError
        
        
        # define the trainer
        batchwise_trainer_flops = BatchwiseTrainer(
            batch_size=cfg.learner.splade_batch_size,
            sampler=PairwiseSampler,
            lossfn=SoftmaxCrossEntropy(),
            hooks=[flops],
        )

        # hooks for the learner
        if cfg.learner.model == "splade_doc":
            hooks = [
                setmeta(
                    DistributedHook(models=[spladev2.encoder]),
                    True,
                )
            ]
        else:
            hooks = [
                setmeta(
                    DistributedHook(models=[spladev2.encoder, spladev2.query_encoder]),
                    True,
                )
            ]

        # establish the validation listener
        validation = ValidationListener(
            id="bestval",
            dataset=self.ds_val,
            # a retriever which use the splade model to score all the
            # documents and then do the retrieve
            retriever=spladev2.getRetriever(
                self.base_retriever_full,
                cfg.full_retriever.batch_size_full_retriever,
                PowerAdaptativeBatcher(),
                device=self.device,
            ),
            early_stop=cfg.learner.early_stop,
            validation_interval=cfg.learner.validation_interval,
            metrics={"RR@10": True, "AP": False, "nDCG@10": False},
            store_last_checkpoint=True if cfg.learner.model == "splade_doc" else False,
        )

        # the learner: Put the components together
        learner = Learner(
            # Misc settings
            random=self.random,
            device=self.device,
            # How to train the model
            trainer=batchwise_trainer_flops,
            # the model to be trained
            scorer=spladev2.tag("model", "splade-v2"),
            # Optimization settings
            optimizers=self.get_optimizers(cfg.learner),
            steps_per_epoch=cfg.learner.steps_per_epoch,
            use_fp16=True,
            max_epochs=tag(cfg.learner.max_epochs),
            # the listener for the validation
            listeners=[validation],
            # the hooks
            hooks=hooks,
        )

        # submit the learner and build the symbolique link
        outputs = learner.submit(launcher=self.gpu_launcher_learner)
        self.tb.add(learner, learner.logpath)

        # get the trained model
        trained_model = (
            outputs.listeners["bestval"]["last_checkpoint"]
            if cfg.learner.model == "splade_doc"
            else outputs.listeners["bestval"]["RR@10"]
        )

        # build a retriever for the documents
        sparse_index = SparseRetrieverIndexBuilder(
            batch_size=16,
            batcher=PowerAdaptativeBatcher(),
            encoder=DenseQueryEncoder(scorer=trained_model),
            device=self.device,
            documents=self.documents,
            ordered_index=False,
        ).submit(launcher=self.gpu_launcher_index)
        
        jobmonitor(sparse_index)
        
        

#@paper_command(schema=SPLADE, package=__package__)
SPLADEIndex(experiment, SPLADE_TREC).run()

INFO:xpm:Submitting job Job[19124417b1a856b73072568f25fb3a315218938e07d4b3851527cf6d7dd8eb92]
INFO:xpm:Starting job /home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/xp/ri/jobs/xpmir.datasets.adapters.randomfold/19124417b1a856b73072568f25fb3a315218938e07d4b3851527cf6d7dd8eb92
INFO:xpm:Process started (Process(5868))
INFO:xpm:Submitting job Job[997eebf3cefd37ca643691f7c7c3c87d0a8e88319277b9eeed0db59ee8aae7a9]
INFO:xpm:Starting job /home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/xp/ri/jobs/xpmir.datasets.adapters.randomfold/997eebf3cefd37ca643691f7c7c3c87d0a8e88319277b9eeed0db59ee8aae7a9
INFO:xpm:Process started (Process(5872))


  0%|          | 0.00/100 [00:00<?, ?%/s]

INFO:geventwebsocket.handler:127.0.0.1 - - [2023-04-09 21:16:48] "GET /notifications/19124417b1a856b73072568f25fb3a315218938e07d4b3851527cf6d7dd8eb92/progress?level=0&progress=0 HTTP/1.1" 200 115 0.001435
INFO:geventwebsocket.handler:127.0.0.1 - - [2023-04-09 21:16:48] "GET /notifications/19124417b1a856b73072568f25fb3a315218938e07d4b3851527cf6d7dd8eb92?status=eoj HTTP/1.1" 404 137 0.000969
INFO:geventwebsocket.handler:127.0.0.1 - - [2023-04-09 21:16:48] "GET /notifications/997eebf3cefd37ca643691f7c7c3c87d0a8e88319277b9eeed0db59ee8aae7a9/progress?level=0&progress=0 HTTP/1.1" 200 115 0.000477
INFO:geventwebsocket.handler:127.0.0.1 - - [2023-04-09 21:16:48] "GET /notifications/997eebf3cefd37ca643691f7c7c3c87d0a8e88319277b9eeed0db59ee8aae7a9?status=eoj HTTP/1.1" 404 137 0.000573
INFO:xpm:Processing 0 dependent jobs
INFO:xpm:Processing 0 dependent jobs


RuntimeError: Job did not complete successfully (ERROR).Check the error log /home/gameselo/Bureau/M1_DAC_2022_2023/PLDAC/xp/ri/jobs/xpmir.datasets.adapters.randomfold/19124417b1a856b73072568f25fb3a315218938e07d4b3851527cf6d7dd8eb92/randomfold.err